In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import filepaths
import schedule_methods
from math import isnan
import math
import datetime
from datetime import timedelta  
from datetime import datetime
import emoji
import re
import warnings
import statistics
from datetime import date
from gsheets import Sheets
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from os.path import isfile, join


warnings.filterwarnings('ignore', '.*apply openpyxl.*', )
warnings.filterwarnings('ignore', '.*Specify dtype option on import.*', )
warnings.filterwarnings('ignore', '.*empty string*', )
warnings.filterwarnings('ignore', '.*invalid value encountered in double_scalars*', )

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',300)
    
def get_lexi():

    print('Loading Lexi Data..')
    lexi = pd.read_excel("/Users/leonarie/Downloads/Lexi 3.0 (1).xlsx")
    lexi['eCPM'] = lexi['Revenue']*1000 / lexi['Delivered']
    lexi['CTR'] = lexi['Clicks']*1000 / lexi['Delivered']
    lexi_clean = lexi.copy()
    print('Lexi Loading Complete!')
    
    #offers = get_offers() #for proper merging
    #lexi = pd.merge(lexi,offers[['Hitpath Offer ID','Day Restrictions','Payout Type','Scheduling Name','Operational Status','Paused PubIDs']],on='Hitpath Offer ID', how='left')
    
    
    #lexi['Affiliate ID'] = lexi['Affiliate ID'].astype(float).astype(int).astype(str) #sometimes Lexi loads with varying dtypes

    #Deal with Duplicate Affiliate IDs for different DP&Pubs
   
    
    return lexi

In [2]:
lexi = get_lexi()
lexi.loc[ lexi['DP&Pub']=='SC.FHA_460398', 'Affiliate ID'] = '160398'
lexi.loc[ lexi['DP&Pub']=='SC.RF_460398', 'Affiliate ID'] = '260398'
lexi.loc[ lexi['DP&Pub']=='LPG.FHA_461128', 'Affiliate ID'] = '161128'
lexi.loc[ lexi['DP&Pub']=='LPG.RF_461128', 'Affiliate ID'] = '261128'

Loading Lexi Data..
Lexi Loading Complete!


In [24]:
lexi['Date'] = pd.to_datetime(lexi['Date'])
lexi = lexi[lexi['Date']>= '2022-01-01']
lexi['Affiliate ID'].astype(str)
lexi['Year'] = lexi['Date'].dt.year
lexi['Month'] = lexi['Date'].dt.month

monthly_data = lexi.groupby(['Affiliate ID', 'Year', 'Month'])['Revenue', 'Delivered', 'Clicks', 'Opens'].sum()


rolling_window = 2
monthly_data['Rolling_Average_Revenue'] = monthly_data.groupby('Affiliate ID')['Revenue'].transform(lambda x: x.rolling(rolling_window, min_periods=1).mean())
monthly_data['Rolling_Average_Delivered'] = monthly_data.groupby('Affiliate ID')['Delivered'].transform(lambda x: x.rolling(rolling_window, min_periods=1).mean())
monthly_data['Rolling_Average_Clicks'] = monthly_data.groupby('Affiliate ID')['Clicks'].transform(lambda x: x.rolling(rolling_window, min_periods=1).mean())
monthly_data['Rolling_Average_Opens'] = monthly_data.groupby('Affiliate ID')['Opens'].transform(lambda x: x.rolling(rolling_window, min_periods=1).mean())


#lexi.info()


#monthly_revenue = lexi.groupby('Affiliate ID','Date')['Revenue'].sum()



#monthly_revenue = lexi.groupby(lexi['Affiliate ID',[lexi['Date'].dt.year, lexi['Date'].dt.month]])['Revenue'].sum()
print(monthly_data)
monthly_data.to_excel("monthly_data.xlsx",
             sheet_name='monthly_data')

<ipython-input-24-c62871a039e4>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  monthly_data = lexi.groupby(['Affiliate ID', 'Year', 'Month'])['Revenue', 'Delivered', 'Clicks', 'Opens'].sum()


                          Revenue  Delivered   Clicks      Opens  \
Affiliate ID Year Month                                            
160398       2022 1      70203.67   14424840  71537.0  3526366.0   
                  2      67388.72   11973848  57131.0  2965557.0   
                  3      66182.27   12226262  57018.0  3128337.0   
                  4      60572.81   12129384  46233.0  2957372.0   
                  5      58495.63   11604668  44085.0  2755809.0   
...                           ...        ...      ...        ...   
461690       2023 5       3309.19     657101   3361.0    62788.0   
461714       2023 5        526.83      49214    512.0    15558.0   
461718       2023 5          0.00      11447     44.0     1209.0   
461719       2023 5        400.89      31318    536.0     5239.0   
461720       2023 5         30.15      21192    340.0     1749.0   

                         Rolling_Average_Revenue  Rolling_Average_Delivered  \
Affiliate ID Year Month             

In [25]:
#Needs weights on 'Revenue_Percent_Change', 'eCPM_Percent_Change', 'CTR_Percent_Change', 'Delivered_Percent_Change'

In [27]:
result = []
monthly_data['Rolling_Average_eCPM'] =  monthly_data['Rolling_Average_Revenue']/monthly_data['Rolling_Average_Delivered']*1000
monthly_data['Rolling_Average_CTR'] =  monthly_data['Rolling_Average_Clicks']/monthly_data['Rolling_Average_Opens']*100


def calculate_revenue_bool(x):
    if x>=10:
        return 0.75
    elif 0 <= x < 10:
        return 0.50
    elif -5 <= x < 0:
        return 0
    elif -10 <= x < -5:
        return -0.50
    else:
        return -0.75
    
def calculate_eCPM_bool(x):
    if x>=0:
        return 0.25
    elif -10 <= x < 0:
        return 0
    else:
        return -0.25
    
def calculate_delivered_bool(x):
    if x>=0:
        return 0.25
    elif -10 <= x < 0:
        return 0
    else:
        return -0.25

def calculate_feedback_bool(x):
    if x >= 0.75:
        return 'High Performing'
    elif 0.50 <= x < 0.75:
        return 'Average Performing'
    elif 0.25 <= x < 0.50:
        return 'Stable Performing'
    else:
        return 'Low Performing'

for affiliate_id, group in monthly_data.groupby('Affiliate ID'):
    # Sort the group by 'Year' and 'Month' to ensure proper ordering
    group = group.sort_values(['Year', 'Month'])
    
    # Exclude the first month
    group = group.iloc[1:]
    
    # Calculate the percentage change
    group['Revenue_Percent_Change'] = group['Rolling_Average_Revenue'].pct_change() * 100
    group['eCPM_Percent_Change'] = group['Rolling_Average_eCPM'].pct_change() * 100
    group['Delivered_Percent_Change'] = group['Rolling_Average_Delivered'].pct_change() * 100
    
    
    group['Revenue_Bool'] = group['Revenue_Percent_Change'].apply(calculate_revenue_bool)
    group['eCPM_Bool'] = group['eCPM_Percent_Change'].apply(calculate_eCPM_bool)
    group['Delivered_Bool'] = group['Delivered_Percent_Change'].apply(calculate_delivered_bool)
    group['Severity Total'] = group['Revenue_Bool'] + group['Delivered_Bool'] + group['eCPM_Bool']
    group['Feedback'] = group['Severity Total'].apply(calculate_feedback_bool)
                                                     
    # Append the results
    result.append(group)

# Concatenate the results into a single DataFrame
result = pd.concat(result)



#monthly_revenue['Revenue % Change'] = monthly_revenue['Revenue'].pct_change() * 100
#monthly_revenue['eCPM % Change'] = monthly_revenue['eCPM'].pct_change() * 100
#monthly_revenue['CTR % Change'] = monthly_revenue['CTR'].pct_change() * 100
#monthly_revenue['Delivered % Change'] = monthly_revenue['Delivered'].pct_change() * 100

# Print the DataFrame
print(result)
result.to_excel("monthly_revenue.xlsx",
             sheet_name='monthly_revenue') 

                          Revenue  Delivered   Clicks      Opens  \
Affiliate ID Year Month                                            
160398       2022 2      67388.72   11973848  57131.0  2965557.0   
                  3      66182.27   12226262  57018.0  3128337.0   
                  4      60572.81   12129384  46233.0  2957372.0   
                  5      58495.63   11604668  44085.0  2755809.0   
                  6      54344.80   10018387  37130.0  2436704.0   
...                           ...        ...      ...        ...   
461670       2023 4       1758.65     447323   1934.0    95773.0   
                  5       1043.92     390837   1322.0   125696.0   
461688       2023 5        656.62     136095   1404.0    28456.0   
461689       2023 5      11057.02    1088057  11970.0   399044.0   
461690       2023 5       3309.19     657101   3361.0    62788.0   

                         Rolling_Average_Revenue  Rolling_Average_Delivered  \
Affiliate ID Year Month             